In [1]:
import os
import json
import cv2
import numpy as np

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

import pvlib
from datetime import datetime

# Load Data

## Load 2020 Meteorological data from Solcast

In [2]:
solcast_data = pd.read_csv('./Solcast_2007-2021_Hourly.csv')

# filter data for year 2020
solcast_data = solcast_data[solcast_data['Year'] == 2020]
# filter out some columns
solcast_data = solcast_data[['Year', 'Month', 'Day', 'Hour', 'Azimuth', 'DHI', 'DNI', 'GHI', 'Zenith']]
# sort values by datetime
solcast_data = solcast_data.sort_values(by=['Year', 'Month', 'Day', 'Hour'])

solcast_data.head()

,Year,Month,Day,Hour,Azimuth,DHI,DNI,GHI,Zenith
113928,2020,1,1,0,-143,0,0,0,168
113929,2020,1,1,1,-115,0,0,0,157
113930,2020,1,1,2,-109,0,0,0,143
113931,2020,1,1,3,-108,0,0,0,129
113932,2020,1,1,4,-109,0,0,0,116


## Load roof segment polygons

In [5]:
gdf = gpd.read_file('../[4] Module Fitting/roof_gdf_module_fitting_flatsouth.geojson')
gdf.head()

,index,segment_id,building_id,address,image_id,category_id,num_panels,panel_area,geometry
0,0,90215,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,12,1160,1898.6880,"MULTIPOLYGON (((121.12337 14.66163, 121.12339 ..."
1,1,112141,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",3707,5,888,1453.4784,"POLYGON ((121.12283 14.66167, 121.12283 14.661..."
2,2,53649,way/871081677,"128 Santan, Marikina, Metro Manila, Philippines",1775,13,740,1211.2320,"POLYGON ((121.12561 14.65969, 121.12560 14.659..."
3,3,90212,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,5,734,1201.4112,"POLYGON ((121.12333 14.66191, 121.12333 14.661..."
4,4,19982,way/16827525,"Unit 2128, Riverbank Center, Riverbanks Avenue...",653,3,710,1162.1280,"POLYGON ((121.08306 14.63122, 121.08306 14.631..."


# Generate irradiance orientation computation functions

In [6]:
# azimuth_values = {
#     # WNW
#     3: 292.5, 
#     # N
#     4: 360, 
#     # NNW
#     5: 337.5,
#     # NW
#     6: 315,
#     # W
#     7: 270,
#     # WSW
#     8: 247.5,
#     # SW
#     9: 225,
#     # SSW
#     10: 202.5,
#     # S
#     11: 180,
#     # SSE
#     12: 157.5,
#     # SE
#     13: 135,
#     # ESE
#     14: 112.5,
#     # E
#     15: 90,
#     # ENE
#     16: 67.5,
#     # NE
#     17: 45,
#     # NNE
#     18: 22.5
# }

azimuth_values = {
    3: 292.5, 4: 360, 5: 337.5, 6: 315, 7: 270, 
    8: 247.5, 9: 225, 10: 202.5, 11: 180, 12: 157.5, 
    13: 135, 14: 112.5,15: 90, 16: 67.5, 17: 45, 18: 22.5
}

flat_roof_azimuth_values = {
    "S": 11, # key 11 in azimuth_values
    "SW": 9 # key 9 in azimuth_values
}

def get_surface_azimuth(category_id, flat_roof = "S"):
    if category_id == 2:
        return azimuth_values[flat_roof_azimuth_values[flat_roof]]
    elif category_id > 2 and category_id <= 18:
        return azimuth_values[category_id]
    else:
        return 180

In [7]:
def compute_orientation_irradiance(roof_category_id, 
                                   weather_data, 
                                   flat_roof_configuration, 
                                   surface_tilt = 27.31, 
                                   flat_surface_tilt = 15):
    
    year = int(weather_data['Year'])
    month = int(weather_data['Month'])
    day = int(weather_data['Day'])
    hour = int(weather_data['Hour'])

    solar_azimuth = weather_data['Azimuth']
    solar_zenith = weather_data['Zenith']
    DHI = weather_data['DHI']
    DNI = weather_data['DNI']
    GHI = weather_data['GHI']

    # for zero value cases
    if DHI == 0 and DNI == 0 and GHI == 0:
        return 0.0

    if roof_category_id == 2:
        surface_tilt = flat_surface_tilt

    # get surface azimuth
    surface_azimuth = get_surface_azimuth(roof_category_id, flat_roof_configuration)

    # get extra-terrestrial radiation
    datetime_str = str(month) + '/' + str(day) + '/' + str(year)[2:] + ' ' + str(hour)
    datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H')
    dni_extra = pvlib.irradiance.get_extra_radiation(datetime_or_doy=datetime_object)

    surface_type = 'urban'

    # call pvlib function
    try:
        total_irradiance = pvlib.irradiance.get_total_irradiance(surface_tilt=surface_tilt,
                                                                surface_azimuth=surface_azimuth,
                                                                solar_zenith=solar_zenith,
                                                                solar_azimuth=solar_azimuth,
                                                                dni=DNI,
                                                                ghi=GHI,
                                                                dhi=DHI,
                                                                dni_extra=dni_extra,
                                                                airmass=None,
                                                                # albedo=,
                                                                surface_type=surface_type,
                                                                model='perez'
                                                                )

        return total_irradiance['poa_global'] / 1000

    except ZeroDivisionError:
        print('Cannot devide by zero.')
        return 0

# Compute hourly irradiance for each orientation

In [8]:
flat_roof_configuration = "S"

# generate list of roof category id's (2-18)
roof_category_ids = np.arange(2, 19, dtype=int)

for catid in roof_category_ids:
  column_name = 'irradiance_catid_' + str(catid)
  print(column_name)
  solcast_data[column_name] = solcast_data.apply(
      lambda x: compute_orientation_irradiance(
          roof_category_id=catid, 
          weather_data=x, 
          flat_roof_configuration=flat_roof_configuration), 
      axis=1)

solcast_data.to_csv('irradiance_orientation_flatsouth.csv', index=False)

irradiance_catid_2


/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/pvlib/irradiance.py:1098: RuntimeWarning: divide by zero encountered in long_scalars
  eps = ((dhi + dni) / dhi + kappa * (z ** 3)) / (1 + kappa * (z ** 3))


irradiance_catid_3


/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/pvlib/irradiance.py:1098: RuntimeWarning: divide by zero encountered in double_scalars
  eps = ((dhi + dni) / dhi + kappa * (z ** 3)) / (1 + kappa * (z ** 3))


irradiance_catid_4
irradiance_catid_5
irradiance_catid_6
irradiance_catid_7
irradiance_catid_8
irradiance_catid_9
irradiance_catid_10
irradiance_catid_11
irradiance_catid_12
irradiance_catid_13
irradiance_catid_14
irradiance_catid_15
irradiance_catid_16
irradiance_catid_17
irradiance_catid_18


# Compute hourly irradiance for each orientation (hypothesis testing)

In [11]:
solcast_data = pd.read_csv('./Solcast_2007-2021_Hourly.csv')

# filter data for years 2007-2018
solcast_data = solcast_data[solcast_data['Year'] <= 2018]
solcast_data = solcast_data[solcast_data['Year'] >= 2007]

# filter out some columns
solcast_data = solcast_data[['Year', 'Month', 'Day', 'Hour', 'Azimuth', 'DHI', 'DNI', 'GHI', 'Zenith']]
# sort values by datetime
solcast_data = solcast_data.sort_values(by=['Year', 'Month', 'Day', 'Hour'])

solcast_data.head()

,Year,Month,Day,Hour,Azimuth,DHI,DNI,GHI,Zenith
0,2007,1,2,0,-145,0,0,0,167
1,2007,1,2,1,-117,0,0,0,157
2,2007,1,2,2,-110,0,0,0,143
3,2007,1,2,3,-108,0,0,0,129
4,2007,1,2,4,-109,0,0,0,116


In [13]:
flat_roof_configuration = "S"

for catid in roof_category_ids:
    column_name = 'irradiance_catid_' + str(catid)
    print(column_name)
    irradiance_orientation_df[column_name] = solcast_data.apply(
        lambda x: compute_orientation_irradiance(roof_category_id=catid,
                                                 weather_data=x, 
                                                 flat_roof_configuration=flat_roof_configuration),
        axis=1)

irradiance_catid_2


/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/pvlib/irradiance.py:1098: RuntimeWarning: divide by zero encountered in long_scalars
  eps = ((dhi + dni) / dhi + kappa * (z ** 3)) / (1 + kappa * (z ** 3))


NameError: name 'irradiance_orientation_df' is not defined

In [ ]:
irradiance_orientation_df_2007_2018 = irradiance_orientation_df.copy()
irradiance_orientation_df_2007_2018 = irradiance_orientation_df_2007_2018.groupby(['Month', 'Day', 'Hour']).mean().reset_index()
irradiance_orientation_df_2007_2018 = irradiance_orientation_df_2007_2018.drop(columns=['Year'])

irradiance_orientation_df_2007_2018.to_csv('irradiance_orientation_2007_2018_flatsouth.csv', index=False)